In [ ]:
pip install -U lightautoml

# Step 0.1. Import necessary libraries 

In [ ]:
# Standard python libraries
import os
import time
import re

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

# Imports from our package
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task

# Step 0.2. Parameters 

In [ ]:
N_THREADS = 4 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 3 * 3600 # Time in seconds for automl run
TARGET_NAME = 'target'

# Step 0.3. Data load 

In [ ]:
%%time

train_data = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
train_data[TARGET_NAME] = train_data[TARGET_NAME].str.slice(start=6).astype(int) - 1
train_data.head()

In [ ]:
test_data = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
test_data.head()

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-may-2021/sample_submission.csv')
submission.head()

# Step 0.5. Add new features

In [ ]:
def create_gr_feats(data):
    # If you find new features, please add here!
    pass
    

all_df = pd.concat([train_data, test_data]).reset_index(drop = True)
create_gr_feats(all_df)
train_data, test_data = all_df[:len(train_data)], all_df[len(train_data):]
print(train_data.shape, test_data.shape)

In [ ]:
train_data.head()

# ========= AutoML preset usage =========


## Step 1. Create Task

In [ ]:
%%time

task = Task('multiclass',)

## Step 2. Setup columns roles

In [ ]:
%%time

roles = {
    'target': TARGET_NAME,
    'drop': ['id'],
}

## Step 3. Train on full data 

In [ ]:
%%time 

automl = TabularUtilizedAutoML(task = task, 
                               timeout = TIMEOUT,
                               cpu_limit = N_THREADS,
                               reader_params = {'n_jobs': N_THREADS},
                               configs_list=[
                                   '../input/lightautoml-configs/conf_0_sel_type_0.yml',
                                   '../input/lightautoml-configs/conf_1_sel_type_1.yml'
                               ])
oof_pred = automl.fit_predict(train_data, roles = roles)
print('oof_pred:\n{}\nShape = {}'.format(oof_pred[:10], oof_pred.shape))

In [ ]:
%%time

# Fast feature importances calculation
fast_fi = automl.get_feature_scores('fast', silent = False)
fast_fi.set_index('Feature')['Importance'].plot.bar(figsize = (20, 10), grid = True)

## Step 4. Predict for test data and check OOF score

In [ ]:
%%time

test_pred = automl.predict(test_data)
print('Prediction for test data:\n{}\nShape = {}'.format(test_pred[:10], test_pred.shape))

print('Check scores...')
print('OOF score: {}'.format(log_loss(train_data[TARGET_NAME].values, oof_pred.data)))

## Step 5. Prepare submission

In [ ]:
submission.iloc[:, 1:] = test_pred.data
submission.to_csv('lightautoml_2lvl_3hours_2configs.csv', index = False)

In [ ]:
submission